<b>Importing Libraries</b>

In [2]:
import pickle
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv(".env")

True

<b>Intialize your llm </b>

In [4]:
llm = AzureChatOpenAI(
    azure_deployment="gpt432",
    openai_api_key=os.getenv('CHAT_API_KEY'),
    azure_endpoint= os.getenv('CHAT_ENDPOINT'),
    openai_api_version=os.getenv("CHAT_API_VERSION")
    
)

In [19]:
from langchain.chains import LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate

In [57]:
memory = ConversationSummaryBufferMemory(llm= llm, memory_key="chat_history", max_token_limit=100, input_key='human_input', return_messages=True)

In [58]:
template = """You are an assistant for question-answering tasks. \
Given a chat history and the latest user question which might reference context in the chat extract_memory \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\
chat_extract_memory: {chat_history}
user_question: {human_input}
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)

In [59]:
llm_chain = LLMChain(
    llm=llm,
    prompt= prompt,
    verbose=True,
    memory=memory,
)

In [60]:
llm_chain.predict(human_input="How about telling me a joke to start things off?")



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Given a chat history and the latest user question which might reference context in the chat extract_memory Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.chat_extract_memory: []
user_question: How about telling me a joke to start things off?


> Finished chain.


"Sure, here's a light-hearted joke for you: Why don't scientists trust atoms? Because they make up everything!"

In [61]:
llm_chain.predict(human_input=" Can you provide information about famous scientists?")



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Given a chat history and the latest user question which might reference context in the chat extract_memory Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.chat_extract_memory: [HumanMessage(content='How about telling me a joke to start things off?'), AIMessage(content="Sure, here's a light-hearted joke for you: Why don't scientists trust atoms? Because they make up everything!")]
user_question:  Can you provide information about famous scientists?


> Finished chain.


'Absolutely, there are many famous scientists across different fields. For example, Albert Einstein is renowned for his theory of relativity in physics. In biology, Charles Darwin is known for his evolutionary theories. Marie Curie made significant contributions to physics and chemistry and is the only person to win a Nobel prize in both disciplines.'

In [62]:
llm_chain.predict(human_input=" what can you tell me about marie curie")



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Given a chat history and the latest user question which might reference context in the chat extract_memory Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.chat_extract_memory: [SystemMessage(content='The human asks the AI to tell a joke to start things off. The AI responds with a light-hearted joke about scientists not trusting atoms because they make up everything.'), HumanMessage(content=' Can you provide information about famous scientists?'), AIMessage(content='Absolutely, there are many famous scientists across different fields. For example, Albert Einstein is renowned for his theory of relativity in physics. In biology, Charles Darwin is known for his evolutionary theories. Marie Curie made significant contributions to physics and chem

'Marie Curie was a renowned physicist and chemist. She is well known for her research on radioactivity, and she was the first woman to win a Nobel Prize, the only woman to win in two fields, and the only person to win in multiple sciences. Her significant contributions were in the fields of Physics and Chemistry.'

<b>Extracting and Passing memory to new chain </b>

In [63]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content='The human asks the AI to tell a joke and then requests information about famous scientists. The AI responds with a joke about atoms and then provides examples of famous scientists like Albert Einstein, Charles Darwin, and Marie Curie, highlighting their significant contributions to their respective fields.'),
  HumanMessage(content=' what can you tell me about marie curie'),
  AIMessage(content='Marie Curie was a renowned physicist and chemist. She is well known for her research on radioactivity, and she was the first woman to win a Nobel Prize, the only woman to win in two fields, and the only person to win in multiple sciences. Her significant contributions were in the fields of Physics and Chemistry.')]}

In [64]:
extract_mem = memory.load_memory_variables({})


If you wish to store and retrieve in database you can use pickle

passing to new chain

In [66]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [67]:
blank_memory = ConversationSummaryBufferMemory(llm = llm, 
                                             max_token_limit=200, 
                                             memory_key='chat_extract_memory', 
                                             input_key='human_input', 
                                             return_messages=True)


In [68]:
blank_memory.load_memory_variables({})

{'chat_extract_memory': []}

In [69]:
for item in extract_mem['chat_history']:
    if isinstance(item, SystemMessage):
        blank_memory.moving_summary_buffer = item.content
    elif isinstance(item, HumanMessage):
        blank_memory.chat_memory.add_user_message(item.content)
    elif isinstance(item, AIMessage):
        blank_memory.chat_memory.add_ai_message(item.content)
 

In [70]:
blank_memory.load_memory_variables({})

{'chat_extract_memory': [SystemMessage(content='The human asks the AI to tell a joke and then requests information about famous scientists. The AI responds with a joke about atoms and then provides examples of famous scientists like Albert Einstein, Charles Darwin, and Marie Curie, highlighting their significant contributions to their respective fields.'),
  HumanMessage(content=' what can you tell me about marie curie'),
  AIMessage(content='Marie Curie was a renowned physicist and chemist. She is well known for her research on radioactivity, and she was the first woman to win a Nobel Prize, the only woman to win in two fields, and the only person to win in multiple sciences. Her significant contributions were in the fields of Physics and Chemistry.')]}